In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import cross_validation

In [7]:
input_file = "income_data.txt"

In [8]:
#data load
X = []
Y = []
count_class1 = 0 
count_class2 = 0
max_datapoints = 25000

In [12]:
with open(input_file, mode="r") as fp : 
    for line in fp : 
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints : break
        if "?" in line : continue
        data = line[:-1].split(", ")
        if data[-1]=="<=50K" and count_class1 < max_datapoints : 
            X.append(data)
            count_class1 += 1
        if data[-1]==">50K" and count_class2 < max_datapoints : 
            X.append(data)
            count_class2 += 1

In [14]:
X = np.array(X)
label_encoder = []
X_encoded = np.empty(X.shape)

In [39]:
for i, item in enumerate(X[0]) : 
    if item.isdigit() : 
        X_encoded[:, i] = X[:, i]
    else :
        label_encoder.append(preprocessing.LabelEncoder()) 
        #preprocessing.LabelEncoder() 인스턴스 객체를 생성해서 list에 추가한다.
        #이 때 Label_Encoder() 객체는 구분할 필요 없는 동일한 객체며, categorical 클래스를 numeric으로 바꿀 회수를 카운트하기 위한 용도
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i]) #fit_transform을 이용해 각각의 숫자에 매핑한다.

In [40]:
X = X_encoded[:, :-1].astype(int)
Y = X_encoded[:, -1].astype(int)

In [42]:
classifier = OneVsOneClassifier(LinearSVC(random_state=0))
classifier.fit(X, Y) 

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [43]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=5)
classifier = OneVsOneClassifier(LinearSVC(random_state=0))
classifier.fit(X_train, Y_train)
Y_test_pred = classifier.predict(X_test)

In [44]:
f1 = cross_validation.cross_val_score(classifier, X, Y, scoring="f1_weighted", cv=3)
print("F1 score : "+str(round(100*f1.mean(),2))+"%")

F1 score : 63.92%


In [46]:
#test data to predict
input_data = ['37', 'Private', '215646', 'HS-grad', '9', 'Never-married', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', '0', '0', '40', 'United-States']

In [82]:
input_data_encoded = [-1]*len(input_data)
count = 0 
for i, item in enumerate(input_data) : 
    if item.isdigit() :
        input_data_encoded[i] = int(input_data[i])
    else :
        input_data_encoded[i] = int(label_encoder[count].transform(input_data[i]))
        count+=1
input_data_encoded = np.array(input_data_encoded)

ValueError: bad input shape ()

In [84]:
predicted_class = classifier.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(predicted_class)[0])

<=50K


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
